---


# merge

### 위의 시트에서 사본시트에서 코드를 확인하여 일치하는 보험 청구 항목을 새로운 열로 추가하는 코드입니다

In [ ]:
import gspread
import re
import time
from oauth2client.service_account import ServiceAccountCredentials

# 구글 스프레드시트 인증 및 설정
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

doc = gc.open_by_url(spreadsheet_url)

insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()

# 추가된 열의 이름 저장
disease_code_sheet.update("C1:D1", [["청구 카테고리", "세부 청구 항목"]])

# 상병 코드를 추적하여 중복될 경우 항목을 병합할 딕셔너리 생성
disease_code_tracker = {}

for disease_row_idx, disease_row in enumerate(disease_code_data[1:], start=2):
    disease_code = disease_row[0]

    # 상병 코드에 '~'가 포함된 경우 빈칸으로 처리
    if not disease_code.strip() or "~" in disease_code:
        clean_category = ""
        clean_item = ""
    else:
        category = "일치하는 보험 상품이 없습니다"
        item = "일치하는 보험 상품이 없습니다"
        matched = False  # 매칭 여부를 추적하는 변수

        for claim_row in insurance_claim_data[1:]:
            claim_codes = claim_row[3].split(",")

            if disease_code in [code.strip() for code in claim_codes]:
                # 숫자를 제거하지 않고 항목 그대로 사용
                new_category = claim_row[0].strip()
                new_item = claim_row[1].strip()

                matched = True  # 매칭이 발생했음을 표시

                # 중복된 상병 코드를 처리하여 병합
                if disease_code in disease_code_tracker:
                    clean_category, clean_item = disease_code_tracker[disease_code]

                    # 중복된 항목 병합
                    if new_category not in clean_category:
                        clean_category += f" | {new_category}"
                    if new_item not in clean_item:
                        clean_item += f" | {new_item}"
                else:
                    clean_category = new_category
                    clean_item = new_item

                # 병합된 결과를 딕셔너리에 저장
                disease_code_tracker[disease_code] = (clean_category, clean_item)

        # 매칭된 코드가 없으면 '일치하는 보험 상품이 없습니다' 추가
        if not matched:
            clean_category = "일치하는 정보가 없습니다"
            clean_item = "일치하는 정보가 없습니다"

    # 병합된 데이터를 시트에 업데이트
    disease_code_sheet.update(
        f"C{disease_row_idx}:D{disease_row_idx}", [[clean_category, clean_item]]
    )
    time.sleep(2)